In [ ]:
import torch
import numpy as np
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

In [ ]:
def part1PlotBoundary(X, y, nn):
    """
    Plots the decision boundary for a given neural network model.

    Parameters:
    - X (torch.Tensor): Input features, shape (n_samples, 2).
    - y (torch.Tensor): Ground truth labels, shape (n_samples,).
    - nn (torch.nn.Module): Trained neural network model.

    Returns:
    - None: Displays a decision boundary plot.
    """

    # Step size for the mesh grid
    h = 0.01

    # Define the grid boundaries
    x_min, x_max = X[:, 0].min().cpu().item() - 0.1, X[:, 0].max().cpu().item() + 0.1
    y_min, y_max = X[:, 1].min().cpu().item() - 0.1, X[:, 1].max().cpu().item() + 0.1

    # Create a mesh grid
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # Generate predictions over the grid
    grid_points = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32).to(device)
    with torch.no_grad():  # Disable gradient computation
        Z = nn(grid_points)  # Forward pass
    Z = Z.detach().cpu().numpy()  # Move to CPU and convert to NumPy
    Z = Z.reshape(xx.shape)  # Reshape to match the grid

    # Plot the filled contour for the decision regions
    plt.contourf(xx, yy, Z, levels=[0, 0.5, 1], colors=['lightblue', 'lightcoral'], alpha=0.5)

    # Add contour line for the decision boundary
    plt.contour(xx, yy, Z, levels=[0.5], colors='black')

    # Plot data points
    plt.scatter(X[:, 0].cpu().numpy(), X[:, 1].cpu().numpy(), c=y.cpu().numpy().flatten(), cmap=plt.cm.RdYlBu, edgecolor='k')

    # Labels and title
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.title('Decision Boundary')
    plt.show()


In [ ]:
#DEFINE YOUR DEVICE
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device) #if cpu, go Runtime-> Change runtime type-> Hardware accelerator GPU -> Save -> Redo previous steps

In [ ]:
#CREATE A RANDOM DATASET
centers = [[1, 1], [1, -1], [-1, -1], [-1, 1]] #center of each class
cluster_std=0.4 #standard deviation of random gaussian samples

x_train, y_train = make_blobs(n_samples=1000, centers=centers, n_features=2, cluster_std=cluster_std, shuffle=True)
y_train[y_train==2] = 0 #make this an xor problem
y_train[y_train==3] = 1 #make this an xor problem
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

x_val, y_val = make_blobs(n_samples=100, centers=centers, n_features=2, cluster_std=cluster_std, shuffle=True)
y_val[y_val==2] = 0 #make this an xor problem
y_val[y_val==3] = 1 #make this an xor problem
x_val = torch.FloatTensor(x_val)
y_val = torch.FloatTensor(y_val)

In [ ]:
#CHECK THE BLOBS ON XY PLOT
plt.scatter(x_train[y_train==0,0],x_train[y_train==0,1],marker='o',color='blue')
plt.scatter(x_train[y_train==1,0],x_train[y_train==1,1],marker='o',color='red')

In [ ]:
#DEFINE NEURAL NETWORK MODEL
class FullyConnected(torch.nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(FullyConnected, self).__init__()
    self.input_size = input_size
    self.hidden_size  = hidden_size
    self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
    self.fc2 = torch.nn.Linear(self.hidden_size, num_classes)
    self.relu = torch.nn.ReLU()
    self.sigmoid = torch.nn.Sigmoid()
  def forward(self, x):
    hidden = self.fc1(x)
    relu = self.relu(hidden)
    output = self.fc2(relu)
    return output

In [ ]:
class FullyConnected2(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
      super(FullyConnected2, self).__init__()
      self.input_size = input_size
      self.hidden_size  = hidden_size
      self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
      self.fc2 = torch.nn.Linear(self.hidden_size, self.hidden_size*4,)
      self.fc3 = torch.nn.Linear(self.hidden_size*4, self.hidden_size*2,)
      self.fc4 = torch.nn.Linear(self.hidden_size*2, num_classes)
      self.relu = torch.nn.ReLU()
      self.sigmoid = torch.nn.Sigmoid()
    def forward(self, x):
      hidden = self.fc1(x)
      relu = self.relu(hidden)
      hidden2 = self.fc2(relu)
      relu2 = self.relu(hidden2)
      hidden3 = self.fc3(relu2)
      relu3 = self.relu(hidden3)
      output = self.fc4(relu3)
      return output

In [ ]:
# Fully Connected Network for Question 5
class FullyConnected3(torch.nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(FullyConnected3, self).__init__()
    self.input_size = input_size
    self.hidden_size  = hidden_size
    self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
    self.fc2 = torch.nn.Linear(self.hidden_size, num_classes)
    self.relu = torch.nn.ReLU()
    self.sigmoid = torch.nn.Sigmoid()
    self.dropout = torch.nn.Dropout(0.5)
  def forward(self, x):
    hidden = self.fc1(x)
    relu = self.relu(hidden)
    dropped = self.dropout(relu)
    output = self.fc2(dropped)
    return output

In [ ]:
#CREATE MODEL
input_size = 2
hidden_size = 64
num_classes = 1

model = FullyConnected(input_size, hidden_size, num_classes)
model.to(device)

In [ ]:
#DEFINE LOSS FUNCTION AND OPTIMIZER
learning_rate = 0.001
momentum = 0.99

loss_fun = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum = momentum)

In [ ]:
#TRAIN THE MODEL
model.train()
epoch = 500
x_train = x_train.to(device)
y_train = y_train.to(device)

loss_values = np.zeros(epoch)

for i in range(epoch):
    optimizer.zero_grad()
    y_pred = model(x_train)    # forward
    #reshape y_pred from (n_samples,1) to (n_samples), so y_pred and y_train have the same shape
    y_pred = y_pred.reshape(y_pred.shape[0])

    loss = loss_fun(y_pred, y_train)

    loss_values[i] = loss.item()
    print('Epoch {}: train loss: {}'.format(i, loss.item()))
    loss.backward() #backward
    optimizer.step()

In [ ]:
#PLOT THE LEARNING CURVE
plt.plot(loss_values)
plt.title('Learning Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid('on')

In [ ]:
#TEST THE MODEL
model.eval()

x_val = x_val.to(device)
y_val = y_val.to(device)

y_pred = model(x_val)
#reshape y_pred from (n_samples,1) to (n_samples), so y_pred and y_val have the same shape
y_pred = y_pred.reshape(y_pred.shape[0])
after_train = loss_fun(y_pred, y_val)
print('Validation loss after Training' , after_train.item())

correct=0
total=0
for i in range(y_pred.shape[0]):
  if y_val[i]==torch.round(y_pred[i]):
    correct += 1
  total +=1

print('Validation accuracy: %.2f%%' %((100*correct)//(total)))

In [ ]:
# Similarly for validation data
x_val = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32).to(device)
part1PlotBoundary(x_val, y_val, model)